In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [2]:
raw_train_set , raw_valid_set , raw_test_set = tfds.load(
name = "imdb_reviews" , 
split = ["train[:90%]" , "train[:90%]" , "test"] , 
as_supervised = True)

In [3]:
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32)#.prefetch(1)
valid_set = raw_valid_set.batch(32)#.prefetch(1)
test_set = raw_test_set.batch(32)#.prefetch(1)

In [4]:
for review , label in raw_train_set.take(2):
    print(review.numpy().decode("utf-8"))
    print("Label:" , label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

In [5]:
vocab_size = 1000
text_vec_layer = keras.layers.TextVectorization(max_tokens = vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews , labels: reviews))

In [6]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
text_vec_layer,
tf.keras.layers.Embedding(vocab_size, embed_size , mask_zero = True), # neglect padding 
tf.keras.layers.GRU(128 , dropout = 0.2), # neglecting padding makes it not forget
tf.keras.layers.Dense(1, activation="sigmoid")
])

In [7]:
# too slow and not worth
model.compile(loss="binary_crossentropy", optimizer="nadam",
metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=3)

Epoch 1/3



704/704 [==============================] - 923s 1s/step - loss: 0.4890 - accuracy: 0.7530 - val_loss: 0.4465 - val_accuracy: 0.8212
Epoch 2/3
704/704 [==============================] - 866s 1s/step - loss: 0.3420 - accuracy: 0.8581 - val_loss: 0.2772 - val_accuracy: 0.8901
Epoch 3/3
704/704 [==============================] - 808s 1s/step - loss: 0.2946 - accuracy: 0.8804 - val_loss: 0.2540 - val_accuracy: 0.9006


In [8]:
reviews = []
labels = []
for review , label in raw_test_set.take(-1):
    reviews.append(review.numpy().decode('utf-8'))
    labels.append(label.numpy())

In [9]:
len(labels)

25000

In [10]:
from sklearn.metrics import accuracy_score

In [11]:
labels_hat_probs = model.predict(reviews)
labels_hat = tf.cast(labels_hat_probs > 0.5, tf.int32)
accuracy_score(labels_hat.numpy().flatten() , labels)

782/782 [==============================] - 142s 175ms/step


0.87108

### 87% accuracy on labeling movie reviews!

### EOF